# Merging Datasets

Sometimes we don't have the luxury of a well prepared Kaggle dataset. Sometimes we may have to work with a database dump, with separate files describing separate tables of the database. And we have to stitch them together ourselves. So that's what we're doing today!

Here's a very simple dataset to practise on: http://tiny.cc/studcsv

(If you've downloaded the zip file, you already have them)

Let's find all students that have failed three or more courses and send their parents an email from their teachers.

In [1]:
import pandas as pd

students = pd.read_csv("students.csv")
teachers = pd.read_csv("teachers.csv")
grades1 = pd.read_csv("grades1.csv")
grades2 = pd.read_csv("grades2.csv")
contacts = pd.read_csv("contact.csv")

students.head(2)

id firstname lastname
0  34292    Joshua    Davis
1  34293     Karen   Flores

In [2]:
teachers.head(2)

course        teacher
0  PHYS101   Wayne Walker
1  CHEM101  Albert Taylor

In [3]:
grades1.head(2)

student_id   course grade
0       34292  ENGL101     A
1       34293  ENGL101     A

In [4]:
grades2.head(2)

student_id   course grade
0       34302  DRAM101     F
1       34303  DRAM101     D

In [5]:
contacts.head(2)

student_id parent_contact           phone
0       34292   Samuel Davis  (356) 849-0352
1       34293   Laura Flores  (477) 325-7117

## Concat and append

You've seen append before. We have grades for 2 semesters, and we want them both together

In [15]:
grades1["Semester"] = 1
grades2["Semester"] = 2

grades = pd.concat([grades1, grades2])
grades

student_id   course grade  Semester
0        34292  ENGL101     A         1
1        34293  ENGL101     A         1
2        34294  ENGL101     C         1
3        34295  ENGL101     C         1
4        34296  ENGL101     C         1
..         ...      ...   ...       ...
60       34312  DESN101     E         2
61       34313  DESN101     D         2
62       34314  DESN101     A         2
63       34315  DESN101     A         2
64       34316  DESN101     A         2

[190 rows x 4 columns]

In [7]:
grades1.copy().append(grades2)

student_id   course grade  Semester
0        34292  ENGL101     A         1
1        34293  ENGL101     A         1
2        34294  ENGL101     C         1
3        34295  ENGL101     C         1
4        34296  ENGL101     C         1
..         ...      ...   ...       ...
60       34312  DESN101     E         2
61       34313  DESN101     D         2
62       34314  DESN101     A         2
63       34315  DESN101     A         2
64       34316  DESN101     A         2

[190 rows x 4 columns]

In [16]:
grades = grades.reset_index(drop=True)
grades

student_id   course grade  Semester
0         34292  ENGL101     A         1
1         34293  ENGL101     A         1
2         34294  ENGL101     C         1
3         34295  ENGL101     C         1
4         34296  ENGL101     C         1
..          ...      ...   ...       ...
185       34312  DESN101     E         2
186       34313  DESN101     D         2
187       34314  DESN101     A         2
188       34315  DESN101     A         2
189       34316  DESN101     A         2

[190 rows x 4 columns]

That's better. Not that it would have caused us any issues in this example. Now lets actually merge these dataframes together. Let's put the student information in, merging on the student id.

## Merging horizontally

In [17]:
student_grades = pd.merge(students, grades, left_on="id", right_on="student_id")
student_grades

id firstname  lastname  student_id   course grade  Semester
0    34292    Joshua     Davis       34292  ENGL101     A         1
1    34292    Joshua     Davis       34292  MATH101     A         1
2    34292    Joshua     Davis       34292  PHYS101     D         1
3    34292    Joshua     Davis       34292  MATH102     F         1
4    34292    Joshua     Davis       34292  COMP101     A         1
..     ...       ...       ...         ...      ...   ...       ...
185  34316   Stephen  Martinez       34316  ARTT101     A         1
186  34316   Stephen  Martinez       34316  DESN101     A         1
187  34316   Stephen  Martinez       34316  DRAM101     F         2
188  34316   Stephen  Martinez       34316  ARTT101     A         2
189  34316   Stephen  Martinez       34316  DESN101     A         2

[190 rows x 7 columns]

In [18]:
print(students.shape, grades.shape, student_grades.shape)

(25, 3) (190, 4) (190, 7)


In [19]:
students2 = students.rename({"id": "student_id"}, axis=1)

In [20]:
students_full = students2.merge(contacts, on="student_id")
students_full.head()

student_id firstname  lastname  parent_contact           phone
0       34292    Joshua     Davis    Samuel Davis  (356) 849-0352
1       34293     Karen    Flores    Laura Flores  (477) 325-7117
2       34294     Julia    Walker     Eric Walker  (871) 639-0797
3       34295   Cynthia  Robinson  Chris Robinson  (574) 683-2107
4       34296    Bonnie      Hall    Dorothy Hall  (384) 293-2113

In [21]:
student_grades = pd.merge(students2, grades, on="student_id")
df = pd.merge(student_grades, contacts, on="student_id")
df.head()

student_id firstname lastname   course grade  Semester parent_contact  \
0       34292    Joshua    Davis  ENGL101     A         1   Samuel Davis   
1       34292    Joshua    Davis  MATH101     A         1   Samuel Davis   
2       34292    Joshua    Davis  PHYS101     D         1   Samuel Davis   
3       34292    Joshua    Davis  MATH102     F         1   Samuel Davis   
4       34292    Joshua    Davis  COMP101     A         1   Samuel Davis   

            phone  
0  (356) 849-0352  
1  (356) 849-0352  
2  (356) 849-0352  
3  (356) 849-0352  
4  (356) 849-0352

In [22]:
df = df.merge(teachers, on="course")
df.head()

student_id firstname  lastname   course grade  Semester  parent_contact  \
0       34292    Joshua     Davis  ENGL101     A         1    Samuel Davis   
1       34293     Karen    Flores  ENGL101     A         1    Laura Flores   
2       34294     Julia    Walker  ENGL101     C         1     Eric Walker   
3       34295   Cynthia  Robinson  ENGL101     C         1  Chris Robinson   
4       34296    Bonnie      Hall  ENGL101     C         1    Dorothy Hall   

            phone       teacher  
0  (356) 849-0352  Ernest Green  
1  (477) 325-7117  Ernest Green  
2  (871) 639-0797  Ernest Green  
3  (574) 683-2107  Ernest Green  
4  (384) 293-2113  Ernest Green

In [26]:
df2 = df.loc[df.grade == "F", "student_id"]
method1 = df2.value_counts().rename_axis("student_id").reset_index(name="counts")
method1

student_id  counts
0        34300       4
1        34302       3
2        34303       2
3        34309       2
4        34311       2
5        34313       2
6        34316       2
7        34292       2
8        34294       2
9        34299       2
10       34301       2
11       34304       2
12       34293       1
13       34297       1
14       34314       1
15       34298       1
16       34310       1
17       34306       1

In [28]:
method2 = df.loc[df.grade == "F", ["student_id", "grade"]].groupby("student_id").count().reset_index()
method2.columns = ["student_id", "counts"]
method2

student_id  counts
0        34292       2
1        34293       1
2        34294       2
3        34297       1
4        34298       1
5        34299       2
6        34300       4
7        34301       2
8        34302       3
9        34303       2
10       34304       2
11       34306       1
12       34309       2
13       34310       1
14       34311       2
15       34313       2
16       34314       1
17       34316       2

In [29]:
sids = method1[method1.counts >= 3].student_id
sids.head()

0    34300
1    34302
Name: student_id, dtype: int64

In [30]:
final = students_full[students_full.student_id.isin(sids)]
final

student_id firstname  lastname   parent_contact           phone
8        34300      Mary  Mitchell  Edward Mitchell  (494) 471-1604
10       34302   Dorothy     Green    Russell Green  (749) 344-5744

## Join

What about pd.join? Its a specific version of merge that works solely more on index.

In [31]:
display(students.head(), contacts.head())

id firstname  lastname
0  34292    Joshua     Davis
1  34293     Karen    Flores
2  34294     Julia    Walker
3  34295   Cynthia  Robinson
4  34296    Bonnie      Hall

student_id  parent_contact           phone
0       34292    Samuel Davis  (356) 849-0352
1       34293    Laura Flores  (477) 325-7117
2       34294     Eric Walker  (871) 639-0797
3       34295  Chris Robinson  (574) 683-2107
4       34296    Dorothy Hall  (384) 293-2113

In [32]:
students2 = students.set_index("id")
contacts2 = contacts.set_index("student_id")

students2.join(contacts2)

firstname  lastname   parent_contact           phone
id                                                        
34292    Joshua     Davis     Samuel Davis  (356) 849-0352
34293     Karen    Flores     Laura Flores  (477) 325-7117
34294     Julia    Walker      Eric Walker  (871) 639-0797
34295   Cynthia  Robinson   Chris Robinson  (574) 683-2107
34296    Bonnie      Hall     Dorothy Hall  (384) 293-2113
34297  Gary Lee       Lee         Mary Lee  (914) 452-6366
34298      Ruby  Thompson    Jose Thompson  (253) 419-7702
34299   Heather    Miller     Annie Miller  (864) 583-9615
34300      Mary  Mitchell  Edward Mitchell  (494) 471-1604
34301    Carlos      Ross       Debra Ross  (592) 663-4425
34302   Dorothy     Green    Russell Green  (749) 344-5744
34303      Joan      Wood      Walter Wood  (457) 429-8627
34304      Judy     Jones       Lisa Jones  (773) 266-0693
34305     Nancy    Turner   Barbara Turner  (908) 350-9105
34306      Mark  Phillips  Jeremy Phillips  (600) 628-1550
34307   Gregory  Gonzales    Sara Gonzales  (588) 469-9488
34308     Shawn    Rivera    Joseph Rivera  (759) 260-4879
34309      Lois     James       Mark James  (762) 286-9877
34310   William   Sanders     Ruth Sanders  (249) 216-0443
34311   Barbara  Peterson   James Peterson  (584) 827-4997
34312   Douglas      Cook       Jimmy Cook  (364) 231-5641
34313     Helen     Allen   Lawrence Allen  (607) 356-7119
34314   Melissa     Price       Anne Price  (545) 667-7557
34315   Russell      Reed      Johnny Reed  (708) 871-9155
34316   Stephen  Martinez   Norma Martinez  (783) 439-7854

The other difference is you can join multiple dataframes at once, provided they dont have overlapping columns.

### Recap
* concat
* append
* merge (inner, left, right, outer)
* join